In [197]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta
import time
import os

In [204]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36")
browser = webdriver.Chrome(options=chrome_options)
# browser.close()

### 1. 뉴스 정보(제목, url, 날짜, 분류) 수집

In [ ]:
# start_day와 수집 기간 설정
start_day, end_day = "2017-01-20", "2017-01-26"
date_format = "%Y-%m-%d"
start_day = datetime.strptime(start_day, date_format)
end_day = datetime.strptime(end_day, date_format)

# 209주 반복
for i in range(209):
    news_info = pd.DataFrame()
    url = f'https://www.nytimes.com/search?dropmab=false&endDate={end_day.strftime(date_format)}&query=trump&sort=newest&startDate={start_day.strftime(date_format)}&types=article'
    browser.get(url)
    start_day += timedelta(days=7)
    end_day += timedelta(days=7)

    # 한 주에 검색되는 기사가 100개 이상일 때, 100개로 제한
    article_num = int(browser.find_element(By.CLASS_NAME, 'css-nayoou').text.split(' ')[1])
    if article_num > 100:
        article_num = 100

    # 뉴스 정보 수집
    while len(news_info) < article_num:
        try:
            browser.find_element(By.XPATH, '//*[@id="site-content"]/div/div[2]/div[2]/div/button').click()
        except Exception as e:
            print(f"Error clicking button: {e}")
            break
        time.sleep(4)
        articles = browser.find_elements(By.CLASS_NAME, 'css-1l4w6pd')
        for article in articles[-10:]:
            title = article.find_element(By.CLASS_NAME, 'css-2fgx4k').text
            link = article.find_element(By.TAG_NAME, 'a').get_attribute('href')
            date = article.find_element(By.CLASS_NAME, 'css-17ubb9w').text
            section = article.find_element(By.CLASS_NAME, 'css-myxawk').text
            news_info = pd.concat([news_info, pd.DataFrame([[title, link, date, section]], columns=['title', 'link', 'date', 'section'])], ignore_index=True)
            # 진행도 출력
            print(f'{len(news_info)} / {article_num}')
    # 데이터프레임 전처리 및 저장
    news_info.drop_duplicates(inplace=True)
    news_info.dropna(inplace=True)
    news_info.reset_index(inplace=True)
    news_info.to_csv(f'../collect_data/{start_day}_trump_news.csv')

#### csv파일 통합

In [5]:
# 현재 디렉토리의 csv파일을 통합
path = '.'
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

data_frames = []

for file in csv_files:
    df = pd.read_csv(os.path.join(path, file))
    # 처리된 csv파일 제거
    # os.remove(os.path.join(path, file))
    data_frames.append(df)

# 데이터프레임 전처리 및 저장
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df.drop(columns=['Unnamed: 0', 'index'], inplace=True)
combined_df = combined_df.sort_values(by='date').reset_index(drop=True)
combined_df['date'] = pd.to_datetime(combined_df['date']).dt.strftime('%Y/%m/%d')
combined_df.to_csv('combined_csv.csv', index=False)


### 2. 뉴스 기사 본문 수집

In [205]:
# 카운트를 통해 진행사항 저장
# 코드에 오류가 발생하거나 중지될 때, cnt를 100단위로 다시 선언해주세요
cnt = 1500
data = []
# 뉴스 정보 link열을 통해 페이지에 방문하여 본문 수집
for link in combined_df['link'][cnt:]:
    cnt += 1
    browser.get(link)
    time.sleep(0.5)
    # 본문 수집 코드
    paragraphs = browser.find_elements(By.CLASS_NAME, 'css-at9mc1')
    main_text = ''
    for paragraph in paragraphs:
        main_text += paragraph.text + ' '
    
    data.append({'text': main_text, 'link': link})
    # 진행율
    print(f'{cnt}/{len(combined_df)} \n{cnt/len(combined_df)}')

    # 중간 저장
    if (len(data) % 100) == 0:
        text_df = pd.DataFrame()
        text_df = pd.concat([text_df, pd.DataFrame(data)], ignore_index=True)
        data = []
        text_df.to_csv(f'./text_df{cnt}.csv')

text_df = pd.DataFrame(data)

1501/20870 
0.07192141830378533
1502/20870 
0.07196933397220891
1503/20870 
0.07201724964063248
1504/20870 
0.07206516530905606
1505/20870 
0.07211308097747963
1506/20870 
0.07216099664590321


KeyboardInterrupt: 

In [108]:
# text_combined_df = text_combined_df.drop(columns='Unnamed: 0').drop_duplicates().reset_index(drop=True)
text_combined_df.dropna(inplace=True)

In [146]:
combined_df[combined_df['title'].str.contains('There Will Always Be Change,’ Trump Says as More Personnel Shake-Ups Loom', na=False)]
# combined_df.iloc[[7041], 1].values
# [text_combined_df['text'].str.contains('The Christmas music obsessive is a familiar specimen.')]

,title,link,date,section
5996,"‘There Will Always Be Change,’ Trump Says as M...",https://www.nytimes.com/2018/03/15/us/politics...,2018/03/15,POLITICS


In [172]:
path = '.'
csv_files = [f for f in os.listdir(path) if f.startswith('text')]

data_frames = []

for file in csv_files:
    df = pd.read_csv(os.path.join(path, file))
    data_frames.append(df)

text_combined_df = pd.concat(data_frames, ignore_index=True)

text_combined_df.to_csv('main_text_csv.csv', index=False)

In [178]:
# pd.concat([combined_df, text_combined_df.drop_duplicates().reset_index(drop=True).drop(columns='Unnamed: 0')], join=)
result_df = pd.merge(combined_df, text_combined_df.drop_duplicates().reset_index(drop=True).drop(columns='Unnamed: 0'), on='link', how='inner')


In [184]:
result_df.to_csv('./result.csv')

In [187]:
result_df.describe()

,title,link,date,section,text
count,1500,1500,1500,1500,1487
unique,1494,1500,31,64,1478
top,"Good, Bad and Mad: Andrew Rosenthal on the News",https://www.nytimes.com/2017/01/25/learning/le...,2017/02/02,OPINION,
freq,5,1,81,302,8


In [188]:
result_df.isna().sum()

title       0
link        0
date        0
section     0
text       13
dtype: int64